In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [15]:
# inicio pidiendo el input al usuario y checando que el input sea correcto

# primer input es el tipo de inmueble

input_tipo_inmueble =('casas','departamentos')
choice_tipo_inmueble = input('tipo inmueble (casas, departamentos): ').lower()

while not choice_tipo_inmueble in input_tipo_inmueble:
    
    print('favor de especificar "casas", "departamentos"')
    choice_tipo_inmueble = input('tipo inmueble (casas, departamentos): ').lower()
    
else:
    print('el tipo de inmueble seleccionado es:', choice_tipo_inmueble)
    
    
# segundo input es si es para venta o renta

input_venta_renta = ('venta', 'renta')
choice_venta_renta =  input('venta o renta: ')

while not choice_venta_renta in input_venta_renta:
    
    print('favor de especificar "venta" o "renta"')
    choice_venta_renta =  input('venta o renta: ').lower()
    
else:
    print('la opcion seleccionada es:', choice_venta_renta)

# tercer input es para saber el Estado donde quieren buscar

input_estado = ('distrito-federal', 'estado-de-mexico')
choice_estado =  input('Estado (distrito-federal o estado-de-mexico): ').lower()

while not choice_estado in input_estado:
    
    print('favor de especificar "distrito-federal" o "estado-de-mexico"')
    choice_estado =  input('Estado (distrito-federal o estado-de-mexico): ').lower()
    
else:
    print('la opcion seleccionada es:', choice_estado)
    
# cuarto input es para saber cuantas paginas scrapear y pongo el valor default en 1

pages_to_scrape = ''

while type(pages_to_scrape) != int:
    try:
        pages_to_scrape = int(input('cuantas paginas quieres scrapear?: '))
        
    except ValueError:
        print("favor de introducir un numero entero")

else:
    print('el numero de paginas a scrapear es de:',pages_to_scrape )



# despues reseteo las listas que uso para crear el df para que no traigan valores antiguos

calle = []
colonia = []
municipio = []
estado = []
m2_final = []
descripcion = []
precio_final = []
tipo_final = []

# contruyo las clases para hacer el scraping

class RealEstateSpider:
    """
    Esta clase esta basada en el spider Ironhack que se hizo en uno de los labs.
    Igual que el Ironhack spider a esta clase se le pasan parametros para el scraping.
    Esta clase esta diseñada para scrapear la pagina de inmuebles de Mercado Libre.
    Lo que arroja es un pandas con la informacion encontrada sobre la oferta de inmuebles.
    
    parametros:
    
    url_pattern: patron de regex pattern de la pagina a scrapear
    pages_to_scrape: cuantas paginas se va a scrapear. 
    sleep_interval: tiempo de espera en segundos de espera entre requests. Si <0, los requests no tienen espera.
    content_parser: funcion para extraer el contenido scrapeado.
    
    """
    def __init__(self, url_pattern, pages_to_scrape=10, sleep_interval=-1, content_parser=None):
        self.url_pattern = url_pattern
        self.pages_to_scrape = pages_to_scrape
        self.sleep_interval = sleep_interval
        self.content_parser = content_parser
    
    """
    Scrapea el contenido de la url dada.
    """
    def scrape_url(self, url):
        response = requests.get(url)
        result = self.content_parser(response.content)
        self.output_results(result)
    
    """
    Exporta el contenido y entrega un pandas dataframe
    """
    @classmethod
    def output_results(self, r):
        df = pd.DataFrame(r).transpose()
        self.df = df


    """
   Funcion para inicializar el scrapeo. Usa un for loop para iterar en las paginas a scrapear. 
    """
    def kickstart(self):
        for i in range(1, (self.pages_to_scrape*49), 49):
            self.scrape_url(self.url_pattern % i)


# construyo la funcion para extraer y limpiar la info obtenida en el scraping

def house_parser(content):
    
    """ funcion para extraer y limpirar el html obtenido por el spider"""
    
    soup = BeautifulSoup(content, 'lxml')
    
    precio = soup.select('ol[class="ui-search-layout ui-search-layout--grid"] \
                          span[class="price-tag-fraction"]')
    tipo =  soup.select('span[class="ui-search-item__group__element ui-search-item__subtitle"]')
    ubicacion = soup.select('span[class="ui-search-item__group__element ui-search-item__location"]')
    m2 = soup.select('ul.ui-search-card-attributes.ui-search-item__group__element > li:nth-of-type(1)')
    
    precio_clean = [element.text.strip() for element in precio]
    tipo_clean = [element.text.strip() for element in tipo]
    ubicacion_clean = [element.text.strip() for element in ubicacion]
    m2_clean = [element.text.strip() for element in m2]
    
    ubicacion_splitter(ubicacion_clean)
    m2_splitter(m2_clean)
    price_append(precio_clean)
    tipo_append(tipo_clean)
    
    return precio_final , m2_final, descripcion ,tipo_final, calle, colonia, municipio , estado

# para la limpieza de los strings mas complejos me apoyo creando un par de funciones especializadas
# en limpiar esa informacion con el fin de hacer el codigo mas entendible.

# esta es la funcion para limpiar el texto de la ubicacion

def ubicacion_splitter(content):
    
    """ funcion para limpiar y dividir la ubicación en calle, colonia, municipio (o alcaldia), y
    estado (entidad federativa). lo deja listo para exportarlo al pandas y tener los datos en 
    diferentes columnas para poder analizar.
    
    Se usa dentro de la funcion house_parser"""

    ubicacion_split = [i.split(',') for i in content]
    
    for i in ubicacion_split:
        
        if len(i) >= 4:
            calle.append(i[0])
            colonia.append(i[1])
            municipio.append(i[2])
            estado.append(i[3])
    
        elif len(i) == 3:
            calle.append('')
            colonia.append(i[0])
            municipio.append(i[1])
            estado.append(i[2])
        
        elif len(i) == 2:
            calle.append('')
            colonia.append('')
            municipio.append(i[0])
            estado.append(i[1])
        
        elif len(i) == 1:
            calle.append('')
            colonia.append('')
            municipio.append('')     
            estado.append(i[0])
        
        else:
            calle.append('')
            colonia.append('')
            municipio.append('')     
            estado.append('')

# esta es la funcion para limpiar el texto de los m2
        
def m2_splitter(content):
    
    """ funcion para limpiar la informacion de los m2 ya que viene en un string. regresa 2 listas
        una con los m2 como int y una con un string con informacion sobre esos m2."""
    
    comma_replace = [i.replace(',', '') for i in content]
    m2_split = [element.split(' ') for element in comma_replace]

    for i in m2_split:
        m2_final.append(int(i[0]))
        descripcion.append(i[-1])

        
def price_append(content):
    for i in content:
        precio_final.append(i)
        
        
def tipo_append(content):
    for i in content:
        tipo_final.append(i)
        
url_pattern = f'https://inmuebles.mercadolibre.com.mx/{choice_tipo_inmueble}/{choice_venta_renta}/{choice_estado}/_Desde_%s' 


# Instantiate the IronhackSpider class
my_spider = RealEstateSpider(url_pattern, pages_to_scrape, content_parser=house_parser)

# Start scraping jobs
my_spider.kickstart()
df = my_spider.df
df.columns = ['precio', 'm2', 'tipo_m2', 'tipo_inmueble', 'calle','colonia', 'municipio', 'estado']
df

tipo inmueble (casas, departamentos): casas
el tipo de inmueble seleccionado es: casas
venta o renta: venta
la opcion seleccionada es: venta
Estado (distrito-federal o estado-de-mexico): estado-de-mexico
la opcion seleccionada es: estado-de-mexico
cuantas paginas quieres scrapear?: 10
el numero de paginas a scrapear es de: 10


,precio,m2,tipo_m2,tipo_inmueble,calle,colonia,municipio,estado
0,"6,236,750",178,construidos,Desarrollo,,Lomas Verdes,Naucalpan,Estado De México
1,"22,000,000",530,construidos,Casa en venta,Fuentes Del Sol,Lomas De Tecamachalco,Naucalpan,Estado De México
2,"30,490,000",650,construidos,Casa en venta,Hacienda De La Luz,Interlomas,Huixquilucan,Estado De México
3,"6,950,000",335,construidos,Casa en venta,,Jardines De San Mateo,Naucalpan,Estado De México
4,"15,900,000",1150,construidos,Casa en venta,Camino A Canteras Y San Cristobal Texcaluacan,San Cristóbal Texcalucan,Huixquilucan,Estado De México
...,...,...,...,...,...,...,...,...
475,"1,650,000",113,construidos,Casa en venta,Mz.1 Lt.b San Felipe,San Pedro Totoltepec,Toluca,Estado De México
476,"4,480,000",390,construidos,Casa en venta,Julio Pardiñas 516,Moderna De La Cruz,Toluca,Estado De México
477,"10,200,000",307,construidos,Casa en venta,Puerta Grande,Bosque Esmeralda,Atizapán De Zaragoza,Estado De México
478,"8,800,000",370,construidos,Casa en venta,Antiguo Camino A Chiluca,Lomas De Bellavista,Atizapán De Zaragoza,Estado De México
